## Objectifs du notebook :

**L’application utilisée par le banquier nous envoie les informations suivantes :** 

_{
    "RevolvingUtilizationOfUnsecuredLines": 0.01703559,
    "NumberOfDependents": 1,
    "DebtRatio": 0,
    "age": 42,
    "NumberOfOpenCreditLinesAndLoans": 6,
    "NumberRealEstateLoansOrLines": 1,
    "NumberOfTime30-59DaysPastDueNotWorse": 1,
    "NumberOfTime60-89DaysPastDueNotWorse": 0,
    "NumberOfTimes90DaysLate": 0
}_

- Créez un nouveau notebook, dans lequel vous calculerez une probabilité de défaut de remboursement.
- Committez, et envoyez moi le lien pour que je retrouve facilement
- Faire une prédiction via l'API bigML et la librairie Sklearn

# Via API BigML

### Import des librairies et connection à l'API BigML

In [1]:
from bigml.api import BigML
from bigml.ensemble import Ensemble
import pandas as pd
# from pandas import read_csv

# api = BigML(project='project/5d94a454eba31d46690001d1')
api = BigML()

Initialisation du dataset

In [2]:
df = {"RevolvingUtilizationOfUnsecuredLines": 0,
 "NumberOfDependents": 0,
 "DebtRatio": 0,
 "age": 0,
 "NumberOfOpenCreditLinesAndLoans": 0,
 "NumberRealEstateLoansOrLines": 0,
 "NumberOfTime30-59DaysPastDueNotWorse": 0,
 "NumberOfTime60-89DaysPastDueNotWorse": 0,
 "NumberOfTimes90DaysLate": 0,
 "MonthlyIncome": 0}

### Nouvel input

In [3]:
new_input = {"RevolvingUtilizationOfUnsecuredLines": 0.01703559, 
             "NumberOfDependents": 1, 
             "DebtRatio": 0, 
             "age": 42, 
             "NumberOfOpenCreditLinesAndLoans": 6, 
             "NumberRealEstateLoansOrLines": 1, 
             "NumberOfTime30-59DaysPastDueNotWorse": 1,
             "NumberOfTime60-89DaysPastDueNotWorse": 0,
             "NumberOfTimes90DaysLate": 0}

### Intégration des données dans la data

In [4]:
df.update(new_input)
print(df)

{'RevolvingUtilizationOfUnsecuredLines': 0.01703559, 'NumberOfDependents': 1, 'DebtRatio': 0, 'age': 42, 'NumberOfOpenCreditLinesAndLoans': 6, 'NumberRealEstateLoansOrLines': 1, 'NumberOfTime30-59DaysPastDueNotWorse': 1, 'NumberOfTime60-89DaysPastDueNotWorse': 0, 'NumberOfTimes90DaysLate': 0, 'MonthlyIncome': 0}


In [5]:
df = pd.DataFrame([df])
df

,DebtRatio,MonthlyIncome,NumberOfDependents,NumberOfOpenCreditLinesAndLoans,NumberOfTime30-59DaysPastDueNotWorse,NumberOfTime60-89DaysPastDueNotWorse,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,RevolvingUtilizationOfUnsecuredLines,age
0,0,0,1,6,1,0,0,1,0.017036,42


### Utilisation d'une méthode pour avoir un dataset avec toutes les modifications

In [6]:
import ipynb.fs.defs.GiveMeCredit_SubmissionKaggle_BigML as gmc_sk

df_for_prediction = gmc_sk.change_dataset(df)
df_for_prediction

,DebtRatio,MonthlyIncome,NumberOfDependents,NumberOfOpenCreditLinesAndLoans,NumberOfTime30-59DaysPastDueNotWorse,NumberOfTime60-89DaysPastDueNotWorse,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,RevolvingUtilizationOfUnsecuredLines,age,IncomePerPerson,isOld,MonthlyDebt,MonthlyBalance,DebtPerPerson,BalancePerPerson,NumberOfTime30-89DaysPastDueNotWorse,NumbersOfOpen-NumberRealEstate
Id,,,,,,,,,,,,,,,,,,
0,0,0,1,6,1,0,0,1,0.017036,42,0.0,0,0,0,0.0,0.0,1,5


### Transformation du dataset en list pour pouvoir lui appliquer un modèle et faire la prédiction

In [7]:
input_data = df_for_prediction.to_dict('index')
print(input_data)

new_input_ok = input_data[0]
new_input_ok

{0: {'DebtRatio': 0, 'MonthlyIncome': 0, 'NumberOfDependents': 1, 'NumberOfOpenCreditLinesAndLoans': 6, 'NumberOfTime30-59DaysPastDueNotWorse': 1, 'NumberOfTime60-89DaysPastDueNotWorse': 0, 'NumberOfTimes90DaysLate': 0, 'NumberRealEstateLoansOrLines': 1, 'RevolvingUtilizationOfUnsecuredLines': 0.01703559, 'age': 42, 'IncomePerPerson': 0.0, 'isOld': '0', 'MonthlyDebt': 0, 'MonthlyBalance': 0, 'DebtPerPerson': 0.0, 'BalancePerPerson': 0.0, 'NumberOfTime30-89DaysPastDueNotWorse': 1, 'NumbersOfOpen-NumberRealEstate': 5}}


{'DebtRatio': 0,
 'MonthlyIncome': 0,
 'NumberOfDependents': 1,
 'NumberOfOpenCreditLinesAndLoans': 6,
 'NumberOfTime30-59DaysPastDueNotWorse': 1,
 'NumberOfTime60-89DaysPastDueNotWorse': 0,
 'NumberOfTimes90DaysLate': 0,
 'NumberRealEstateLoansOrLines': 1,
 'RevolvingUtilizationOfUnsecuredLines': 0.01703559,
 'age': 42,
 'IncomePerPerson': 0.0,
 'isOld': '0',
 'MonthlyDebt': 0,
 'MonthlyBalance': 0,
 'DebtPerPerson': 0.0,
 'BalancePerPerson': 0.0,
 'NumberOfTime30-89DaysPastDueNotWorse': 1,
 'NumbersOfOpen-NumberRealEstate': 5}

### Modèle

In [8]:
model_ensemble = Ensemble('ensemble/5dc2d7d3e476847468013866', api=BigML(domain="bigml.io"))

### Prédiction

In [9]:
input_prediction = model_ensemble.predict(new_input_ok, full=True)
input_prediction

{'probability': 0.96507, 'prediction': '0'}

- input_prediction(monthly_0) = 0.96507
- input_prediction(monthly_1000) = 0.96507
- input_prediction(monthly_3000) = 0.96914
- input_prediction(monthly_15000) = 0.98153
- input_prediction(monthly_300000) = 0.98162

# Actions communes pour utilisation des librairies Scikit-learn et XGBoost

Run [GiveMeCredit_Xgboost_Sklearn](GiveMeCredit_Xgboost_Sklearn.ipynb) notebook to create `model_skl` et `model_xgb`

In [10]:
%run GiveMeCredit_Xgboost_Sklearn.ipynb

(150000, 19)
(101503, 19)
(150000, 19)
(101503, 19)
(150000,)
(150000, 18)
(101503, 18)


100%|██████████| 1.70M/1.70M [00:03<00:00, 490kB/s] 
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
100%|██████████| 984k/984k [00:05<00:00, 197kB/s]  


#### Définition de X_new_input pour les 2 modèles

In [15]:
X_new_input = df_for_prediction.values

# Via librairie Sklearn

#### Prédiction pour nouvel input

In [12]:
y_nem_input_skl = model_skl.predict_proba(X_new_input)

In [13]:
y_nem_input_skl

array([[0.7, 0.3]])

# Via librairie XGBoost 

#### Prédiction pour nouvel input

In [16]:
y_nem_input_xgb = model_xgb.predict_proba(X_new_input)

In [17]:
y_nem_input_xgb

array([[0.9511007, 0.0488993]], dtype=float32)